In [1]:
!pip install openmeteo_requests requests_cache retry_requests xgboost gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.4/393.4 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 99.4 MB/s eta 0:00:00


In [15]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
import joblib
from xgboost import XGBRegressor

# Load scaler and model
scaler = joblib.load("scaler.pkl")
xgb = XGBRegressor()
xgb.load_model("xgb_scaled_model.json")

# Function to fetch data & predict AQI
def predict_aqi(lat, lon):
    url = (
        f"https://air-quality-api.open-meteo.com/v1/air-quality"
        f"?latitude={lat}&longitude={lon}"
        f"&hourly=pm10,carbon_monoxide,sulphur_dioxide,aerosol_optical_depth,ozone,dust"
        f"&forecast_days=3"
    )

    response = requests.get(url)
    if response.status_code != 200:
        return pd.DataFrame(), None

    data = response.json()
    if "hourly" not in data:
        return pd.DataFrame(), None

    hourly = data["hourly"]
    df = pd.DataFrame(hourly)

    # Select features
    features = ["pm10", "carbon_monoxide", "sulphur_dioxide", "dust", "aerosol_optical_depth", "ozone"]
    if not all(f in df.columns for f in features):
        return pd.DataFrame(), None

    X = df[features].fillna(0)

    # Scale
    X_scaled = scaler.transform(X)

    # Predict
    y_pred = xgb.predict(X_scaled)

    # Add predictions to DataFrame
    df["predicted_us_aqi"] = y_pred

    # Take daily averages
    df["time"] = pd.to_datetime(df["time"])
    df.set_index("time", inplace=True)
    daily_aqi = df["predicted_us_aqi"].resample("D").mean().reset_index()

    # Plot
    plt.figure(figsize=(8, 4))
    plt.plot(daily_aqi["time"], daily_aqi["predicted_us_aqi"], marker="o", label="Predicted AQI")
    plt.xlabel("Date")
    plt.ylabel("Predicted AQI")
    plt.title("Daily Predicted AQI")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("aqi_plot.png")
    plt.close()

    return daily_aqi, "aqi_plot.png"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🌍 Air Quality Prediction Dashboard (XGBoost + Scaler)")

    with gr.Row():
        lat = gr.Number(label="Latitude", value=33.6844)  # Example: Islamabad
        lon = gr.Number(label="Longitude", value=73.0479)

    predict_btn = gr.Button("Predict AQI")
    output_table = gr.Dataframe(label="Daily AQI Predictions")
    output_plot = gr.Image(label="AQI Visualization", type="filepath")

    predict_btn.click(fn=predict_aqi, inputs=[lat, lon], outputs=[output_table, output_plot])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2a5f39e37732453b7d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
